In [ ]:
try:                 
    get_ipython().magic(u'load_ext autoreload')    
    get_ipython().magic(u'autoreload 2')           
    get_ipython().magic(u'matplotlib qt')          
except:              
    pass             
import logging       
import matplotlib.pyplot as plt                    
import numpy as np   
                     
logging.basicConfig(format= 
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",  
                    level=logging.DEBUG)           
                     
import caiman as cm  
from caiman.source_extraction import cnmf                 
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect        
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2           
                     
try:                 
    cv2.setNumThreads(0)
except:              
    pass             
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()                                                                                                            
      

In [1]:
               
    import glob          
    import re            
    import os

In [5]:
animal_id = '191126' 
resultDir = 'G:\data\miniscope'
tsFileList = glob.glob(os.path.join(r'W:\qiushou\miniscope\2019*'
                                    ,animal_id     
                                    ,"H*/timestamp.dat"))
def sort_key(s):     
    if s:            
        try:         
            date = re.findall('\d{8}', s)[0]
        except:      
            date = -1            
        try:         
            H = re.findall('H(\d+)',s)[0]
        except:      
            H = -1            
        try:         
            M = re.findall('M(\d+)',s)[0]
        except:      
            M = -1            
        try:         
            S = re.findall('S(\d+)',s)[0]
        except:      
            S = -1            
        try:         
            ms = re.findall('msCam(\d+)',s)[0]
        except:      
            ms = -1  
        return [int(date),int(H),int(M),int(S),int(ms)]
tsFileList.sort(key=sort_key)
tsFileList = [i for i in tsFileList ]#if 'H22_M31_S23' not in i

['W:\\qiushou\\miniscope\\20190927\\191126\\H15_M14_S49\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190927\\191126\\H15_M35_S51\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190928\\191126\\H22_M10_S43\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190928\\191126\\H22_M24_S29\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190928\\191126\\H22_M27_S4\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M28_S41\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M34_S9\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M36_S52\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M40_S43\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M42_S53\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190929\\191126\\H15_M44_S57\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190930\\191126\\H14_M33_S34\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190930\\191126\\H14_M36_S40\\timestamp.dat', 'W:\\qiushou\\miniscope\\20190930\\191126\\H14_M45_S16\\timestamp

In [6]:
[print(i) for i in tsFileList]
print(len(tsFileList))
import numpy as np

W:\qiushou\miniscope\20190927\191126\H15_M14_S49\timestamp.dat
W:\qiushou\miniscope\20190927\191126\H15_M35_S51\timestamp.dat
W:\qiushou\miniscope\20190928\191126\H22_M10_S43\timestamp.dat
W:\qiushou\miniscope\20190928\191126\H22_M24_S29\timestamp.dat
W:\qiushou\miniscope\20190928\191126\H22_M27_S4\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M28_S41\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M34_S9\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M36_S52\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M40_S43\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M42_S53\timestamp.dat
W:\qiushou\miniscope\20190929\191126\H15_M44_S57\timestamp.dat
W:\qiushou\miniscope\20190930\191126\H14_M33_S34\timestamp.dat
W:\qiushou\miniscope\20190930\191126\H14_M36_S40\timestamp.dat
W:\qiushou\miniscope\20190930\191126\H14_M45_S16\timestamp.dat
W:\qiushou\miniscope\20190930\191126\H14_M45_S51\timestamp.dat
W:\qiushou\miniscope\20190930\191126\H15_M4_S31\timestamp

In [7]:
import cv2          
import pickle       
resultDir = 'G:\data\miniscope'
newpath_parent=os.path.join(resultDir,'Results'+'_'+animal_id)
newpath = os.path.join(newpath_parent,"20191016_110707_all")

#%% concatenate timestamps of tsFileList
ms_ts_name = os.path.join(newpath,'ms_ts.pkl')
print(ms_ts_name)  
temporal_downsampling=3                   
import pandas as pd
import pickle   

G:\data\miniscope\Results_191126\20191016_110707_all\ms_ts.pkl


In [7]:
# 这一段是原先杨其鑫的代码，每一block会少一帧
if not os.path.exists(ms_ts_name):
    ts_session=[]  
    for tsFile in tsFileList:
        datatemp=pd.read_csv(tsFile,sep = "\t", header = 0)
        ts_session.append(datatemp['sysClock'].values) 
#         print(len((datatemp['sysClock'].values)))
    ttemp=np.hstack(ts_session)[::temporal_downsampling]
    # remporally downsample for each video
    # [i[::3] for i in ts_session][0]
    session_indend=(np.where(np.diff(ttemp)<0)[0]).tolist()    
    session_indend.append(-1)
    ts_session_ds=[]
    i0=0            
    session_indstart=[]
    for i in range(len(ts_session)):
#         print(i)
        session_indstart.append(i0)
        ts_session_ds.append(ttemp[i0:session_indend[i]])
        i0=session_indend[i]+1
    ms_ts=np.array(ts_session_ds)    
    with open(ms_ts_name,'wb') as output:
        pickle.dump(ms_ts,output,pickle.HIGHEST_PROTOCOL)
else:               
    with open(ms_ts_name, "rb") as f:
        ms_ts= pickle.load(f)
print(f'concatenated timestamp of miniscope_video is located at {ms_ts_name}')

l_ms_ts=[]
for i in ms_ts:
    l_ms_ts.append(len(i))
print(sum(l_ms_ts))

IndexError: list index out of range

In [8]:
#这一段是更正后的代码，帧数相等
if not os.path.exists(ms_ts_name):
    ts_session=[]
    for tsFile in tsFileList:
        datatemp=pd.read_csv(tsFile,sep = "\t", header = 0)
        ts_session.append(datatemp['sysClock'].values)    
    ttemp=np.hstack(ts_session)[::temporal_downsampling]
    # remporally downsample for each video
    # [i[::3] for i in ts_session][0]
    session_indend=(np.where(np.diff(ttemp)<0)[0]).tolist()
#    session_indend.append(-1)
    ts_session_ds=[]
    i0=0
    session_indstart=[]
    if len(session_indend)>0:
        for i in range(len(session_indend)):
            session_indstart.append(i0)
            ts_session_ds.append(ttemp[i0:(session_indend[i]+1)])
            i0=session_indend[i]+1
        ts_session_ds.append(ttemp[(session_indend[-1]+1):])
    else:
        ts_session_ds.append(ttemp[i0:])
    
    
    ms_ts=np.array(ts_session_ds)    
    with open(ms_ts_name,'wb') as output:
        pickle.dump(ms_ts,output,pickle.HIGHEST_PROTOCOL)
else:
    with open(ms_ts_name, "rb") as f:
        ms_ts= pickle.load(f)
print(f'concatenated timestamp of miniscope_video is located at {ms_ts_name}')

concatenated timestamp of miniscope_video is located at G:\data\miniscope\Results_191126\20191016_110707_all\ms_ts.pkl


In [9]:
sum([len(i) for i in ms_ts]) 

114660

In [10]:
len(ms_ts)

16

In [30]:
import cv2
for tsFile in tsFileList:
    print(tsFile)
    print(">",end="")
    ts = pd.read_csv(tsFile,sep = "\t", header = 0)
    ts_len=ts.shape[0]        
    videoFileList=glob.glob(os.path.dirname(tsFile)+'\*.avi')   
    framenum=[]
    for video in videoFileList:
#         print(video)
        print("<",end="")
        cap = cv2.VideoCapture(video)
        framenum.append(int(cap.get(7)))
        cap.release()
    print([ts_len,sum(framenum)])

W:\qiushou\miniscope\20190927\191126\H15_M14_S49\timestamp.dat
><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<[71512, 71512]
W:\qiushou\miniscope\20190927\191126\H15_M35_S51\timestamp.dat
><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<[72871, 72871]
W:\qiushou\miniscope\20190928\191126\H22_M10_S43\timestamp.dat
><<<<<<<<<<<<<<<<<<<<<<<<[23875, 23875]
W:\qiushou\miniscope\20190928\191126\H22_M24_S29\timestamp.dat
><<[1733, 1791]
W:\qiushou\miniscope\20190928\191126\H22_M27_S4\timestamp.dat
><<<<<<<<<<<<<<<<<<<<<<<<<[24523, 24523]
W:\qiushou\miniscope\20190929\191126\H15_M28_S41\timestamp.dat
><<<<<<<<<<[9832, 9832]
W:\qiushou\miniscope\20190929\191126\H15_M34_S9\timestamp.dat
><<<<<[4899, 4899]
W:\qiushou\miniscope\20190929\191126\H15_M36_S52\timestamp.dat
><<<<<<[5144, 5144]
W:\qiushou\miniscope\20190929\191126\H15_M40_S43\timestamp.dat
><<<<[3860, 3870]
W:\qiushou\miniscope\20190929\191126\H15_M42_S53\timestamp.dat
><[0, 0]
W:\qiush

SyntaxError: EOL while scanning string literal (<ipython-input-29-2ddba5169899>, line 1)